## Dashboard de Casos do COVID no Brasil com dados de 2020

### Etapas do projeto:

1. Bibliotecas;
2. Dataset;
3. Mapa;
4. Layout;
5.Interatividade.


A principal biblioteca para criação do projeto é _plotly_, pois além de ter possuir vários tipos de gráficos, tabelas e mapas também possui o módulo de dashaboard. A biblioteca _Dash Bootstrap Components_, utilizada para alterar os temas, dando um visual mais amigavel.

A biblioteca _dash_ é responsável pela geração do dashboard, ou seja, criando no servido e o layout. O _dash_core_components_ é o responsável pela criação de todos os componentes dentro do dashboard, como dropdown e buttons. O _dash_html_components_ permite a inclussão do textos html, como _div_, _id_, _class_, entre outros, funcionando como complemento do _dash_core_components_.


In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc 

import plotly.express as px
import plotly.graph_objects as go

import numpy as np
import pandas as pd
import json

#### Solicitação dos dados do covid e exportação para dois arquivos (estados e brasil)

In [3]:
df = pd.read_csv("HIST_PAINEL_COVIDBR_13mai2021.csv", sep=";")
df_states = df[(~df["estado"].isna()) & (df["codmun"].isna())]
df_brazil = df[df["regiao"]=="Brasil"]
df_states.to_csv("df_states.csv")
df_brazil.to_csv("df_brasil.csv")

In [4]:
df_states = pd.read_csv("df_states.csv")
df_brazil = pd.read_csv("df_brasil.csv")

df_data = df_states[df_states["estado"]=="RJ"]

df_states_ = df_states[df_states["data"] == "2020-05-13"]
select_columns = {"casosAcumulado": "Casos Acumulados", 
                "casosNovos": "Novos Casos", 
                "obitosAcumulado": "Óbitos Totais",
                "obitosNovos": "Óbitos por dia"}

#### Leitura do arquivo geojson

In [5]:
brazil_states = json.load(open("geojson/brazil_geo.json", "r"))

## informações do arquivo geojson
#type(brazil_states)
#brazil_states.keys()
#brazil_states["type"]
#brazil_states["features"]
#type(brazil_states["features"])
#brazil_states["features"][0]
#type(brazil_states["features"][0])
#brazil_states["features"][0].keys()
#brazil_states["features"][0]['id']
#brazil_states["features"][0]['geometry']

#### Criação do mapa

In [ ]:
# === Instanciação do dash === #
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

fig = px.choropleth_mapbox(df_states_, locations = "estado", color="casosNovos",
                           center={"lat": -12.984354, "lon": -53.465612}, zoom=4, geojson = brazil_states, 
                           color_continuous_scale = "Redor", opacity = 0.4, 
                           hover_data = {"casosAcumulado": True, "casosNovos": True, "obitosNovos": True, "estado": True})


fig.update_layout(
    paper_bgcolor="#242424",
    autosize=True,
    margin=go.Margin(l=0, r=0, t=0, b=0),
    showlegend = False,
    mapbox_style = "carto-darkmatter"
)


fig2 = go.Figure(layout={"template":"plotly_dark"})
fig2.add_trace(go.Scatter(x=df_data["data"], y=df_data["casosAcumulado"]))
fig2.update_layout(
    paper_bgcolor="#242424",
    plot_bgcolor="#242424",
    autosize=True,
    margin=dict(l=10,r=10,t=10,b=10)
)

# === Construção do Layout === #
app.layout = dbc.Container(  
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Img(id="logo", src=app.get_asset_url("logo_est.png"), height=50),
                html.Span("Por: Letícia de Sousa Borges", style={"margin-top": "25px"}),
                html.H5(children="Evolução COVID-19 no ano de 2020 à 2021"),
                dbc.Button("BRASIL", color="primary", id="location-button", size="lg")
                ], style={}
            ),
            html.P("Informe a data na qual deseja obter inofrmações:", style={"margin-top:":"40px"}),
            html.Div(id="div-test", children=[
                dcc.DatePickerSingle(
                    id="date-picker",
                    min_date_allowed=df_brazil["data"].min(),
                    max_date_allowed=df_brazil["data"].max(),
                    initial_visible_month = df_brazil["data"].min(),
                    date = df_brazil["data"].max(),
                    display_format="MMM D, YYYY",
                    style = {"border": "0px solid black"}
                )
            ]),
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.Span("Casos recuperados"),
                            html.H4(style={"color":"#adfc92"}, id="casos-recuperados-text"),
                            html.Span("Casos acompanhamento"),
                            html.H5(id="em-acompanhamento-text"),
                        ])
                    ], color="light", outline=True, 
                        style={"margin-top":"10px",
                               "box-shadow":"0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0, 0, 0, 0.19)", 
                               "color":"#FFFFFF"})
                ],md=4),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.Span("Casos confirmados totais"),
                            html.H4(style={"color":"#389fd6"}, id="casos-confirmados-text"),
                            html.Span("Novos casos na data"),
                            html.H5(id="novos-casos-text"),
                        ])
                    ], color="light", outline=True, 
                        style={"margin-top":"10px",
                               "box-shadow":"0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0, 0, 0, 0.19)", 
                               "color":"#FFFFFF"})
                ],md=4),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.Span("Óbitos confirmados"),
                            html.H4(style={"color":"#DF2935"}, id="obitos-text"),
                            html.Span("Óbitos na data"),
                            html.H5(id="obitos-na-data-text"),
                        ])
                    ], color="light", outline=True, 
                        style={"margin-top":"10px",
                               "box-shadow":"0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0, 0, 0, 0.19)", 
                               "color":"#FFFFFF"})
                ],md=4)
            ]),
            html.Div([
                html.P("Selecione que tipo de dado deseja vizualizar:", style={"margin-top:":"25px"}),
                dcc.Dropdown(id="location-dropdown", 
                            options=[{"label": j,"value":i} for i, j in select_columns.items()],
                             value="casosNovos",
                             style={"margin-top":"10px"}
                            ),
                dcc.Graph(id="line-graph", figure=fig2, 
                          style={"background-color":"#242424"}),
            ], id ="teste")  
        ],md=5, style={"padding":"25px", "background-color":"#242424"}),
        dbc.Col([
            dcc.Loading(id="loading-1",type="default", 
                        children=[
                            dcc.Graph(id="choropleth-map", figure=fig, style={"height":"104vh", "margin-right":"10px"})
                        ]
                    )
        ], md=7)
    ])
, fluid=True)


# === Interatividade === #
@app.callback(
    [
        Output("casos-recuperados-text", "children"),
        Output("em-acompanhamento-text", "children"),
        Output("casos-confirmados-text", "children"),
        Output("novos-casos-text", "children"),
        Output("obitos-text", "children"),
        Output("obitos-na-data-text", "children")
    ],
    [Input("date-picker", "date"), Input("location-button","children")]
)

def display_status(date, location):
    if location=="BRASIL":
        df_data_on_date = df_brazil[df_brazil["data"]==date]
    else:
        df_data_on_date = df_stats[(df_states["estado"]==location) & (df_states["data"]==date)]
    
    recuperados_novos = "-" if df_data_on_date["Recuperadosnovos"].isna().values[0] else f'{int(df_data_on_date["Recuperadosnovos"].values[0]):,}'.replace(",", ".") 
    acompanhamentos_novos = "-" if df_data_on_date["emAcompanhamentoNovos"].isna().values[0]  else f'{int(df_data_on_date["emAcompanhamentoNovos"].values[0]):,}'.replace(",", ".") 
    casos_acumulados = "-" if df_data_on_date["casosAcumulado"].isna().values[0]  else f'{int(df_data_on_date["casosAcumulado"].values[0]):,}'.replace(",", ".") 
    casos_novos = "-" if df_data_on_date["casosNovos"].isna().values[0]  else f'{int(df_data_on_date["casosNovos"].values[0]):,}'.replace(",", ".") 
    obitos_acumulado = "-" if df_data_on_date["obitosAcumulado"].isna().values[0]  else f'{int(df_data_on_date["obitosAcumulado"].values[0]):,}'.replace(",", ".") 
    obitos_novos = "-" if df_data_on_date["obitosNovos"].isna().values[0]  else f'{int(df_data_on_date["obitosNovos"].values[0]):,}'.replace(",", ".") 
    
    return (
            recuperados_novos, 
            acompanhamentos_novos, 
            casos_acumulados, 
            casos_novos, 
            obitos_acumulado, 
            obitos_novos
            )


@app.callback(
        Output("line-graph", "figure"),
        [Input("location-dropdown", "value"), Input("location-button", "children")]
)
def plot_line_graph(plot_type, location):
    if location == "BRASIL":
        df_data_on_location = df_brazil.copy()
    else:
        df_data_on_location = df_states[df_states["estado"] == location]
     
    bar_plots = ["casosNovos", "obitosNovos"]
    
    fig2 = go.Figure(layout={"template":"plotly_dark"})
    if plot_type in bar_plots:
        fig2.add_trace(go.Bar(x=df_data_on_location["data"], y=df_data_on_location[plot_type]))
    else:
        fig2.add_trace(go.Scatter(x=df_data_on_location["data"], y=df_data_on_location[plot_type]))
    
    fig2.update_layout(
        paper_bgcolor="#242424",
        plot_bgcolor="#242424",
        autosize=True,
        margin=dict(l=10, r=10, b=10, t=10),
        )
    return fig2


@app.callback(
    Output("choropleth-map","figure"),
    [Input("date-picker","date")]
)
def update_map(date):
    df_data_on_states = df_states[df_states["data"]==date]
    
    fig = px.choropleth_mapbox(df_data_on_states, locations = "estado", color="casosAcumulado",
                           center={"lat": -12.984354, "lon": -53.465612}, zoom=4, geojson = brazil_states, 
                           color_continuous_scale = "Redor", opacity = 0.55, 
                           hover_data = {"casosAcumulado": True, "casosNovos": True, "obitosNovos": True, "estado": True})

    fig.update_layout(paper_bgcolor="#242424", autosize=True,margin=go.Margin(l=0, r=0, t=0, b=0),
        showlegend = False, mapbox_style = "carto-darkmatter")
    
    return fig

@app.callback(
    Output("location-button","children"),
    [Input("choropleth-map","clickData"), Input("location-button","n_clicks")]    
)
def update_location(click_data, n_clicks):
    changed_id = [p["prop_id"]for p in dash.callback_context.triggered][0]
    if click_data is not None and changed_id != "location-button.n_clicks":
        state = click_data["points"][0]["location"]
        return "{}".format(state)
    else:
        return "BRASIL"
    
    
if __name__ == "__main__":
    app.run_server(port=8085, host="127.0.0.1")

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

Dash is running on http://127.0.0.1:8085/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8085/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2022 07:27:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 07:27:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 07:27:19] "GET /_favicon.ico?v=2.2.0 HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 07:27:21] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 07:27:22] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 07:27:22] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 07:27:22] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 07:27:22] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 07:27:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 07:27:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 07:27:23] "POST /_dash

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\letic\anaconda3\lib\site-packages\dash\dash.py", line 1344, in dispatch
    response.set_d

127.0.0.1 - - [14/Mar/2022 07:27:58] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\letic\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\letic\anaconda3\lib\site-packages\dash\dash.py", line 1344, in dispatch
    response.set_d

127.0.0.1 - - [14/Mar/2022 07:27:58] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [14/Mar/2022 07:28:02] "POST /_dash-update-component HTTP/1.1" 200 -
